In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Part 1 preparing and analysing Sydney data #

## Lets get the Austrlian data from the csv file ##

In [2]:
australia_df = pd.read_csv('Australian_Post_Codes_Lat_Lon.csv')
australia_df.head()

,postcode,suburb,state,dc,type,lat,lon
0,200,AUSTRALIAN NATIONAL UNIVERSITY,ACT,AUSTRALIAN NATIONAL UNI LPO,Post Office Boxes,-35.277272,149.117136
1,221,BARTON,ACT,NaN,LVR,-35.201372,149.095065
2,800,DARWIN,NT,DARWIN DELIVERY CENTRE,Delivery Area,-12.801028,130.955789
3,801,DARWIN,NT,DARWIN DELIVERY CENTRE,Post Office Boxes,-12.801028,130.955789
4,804,PARAP,NT,PARAP,Post Office Boxes,-12.432181,130.843310


In [3]:
australia_df.shape

(16080, 7)

#### We need to select state of NSW from the Australian dataframe #### 

In [4]:
nsw_df = australia_df.loc[australia_df['state'] == ('NSW')]
nsw_df.head()

,postcode,suburb,state,dc,type,lat,lon
145,1001,SYDNEY,NSW,EAST SUBS MC,LVR,-33.794883,151.268071
146,1002,SYDNEY,NSW,EAST SUBS MC,LVR,-33.794883,151.268071
147,1003,SYDNEY,NSW,EAST SUBS MC,LVR,-33.794883,151.268071
148,1004,SYDNEY,NSW,EAST SUBS MC,LVR,-33.794883,151.268071
149,1005,SYDNEY,NSW,GPO BOX CENTRE,LVR,-33.794883,151.268071


In [5]:
nsw_df.shape

(5050, 7)

#### Next step is to filter the state and get the Sydney metropolitan area ####
Now I will filter the new dataframe to just get the require Latitude and Longitude by using findlattitudelongitude 
service.
https://www.findlatitudeandlongitude.com/

First step is to get the boundary of north of Sydney and suburb from -33.75 to the south should be selected 

In [6]:
nsw1_df = nsw_df.loc[nsw_df['lat'] <= (-33.84)]
nsw1_df.head()

,postcode,suburb,state,dc,type,lat,lon
270,1192,SYDNEY,NSW,NaN,LVR,-34.790684,147.685283
287,1209,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251
288,1210,AUSTRALIA SQUARE,NSW,EAST SUBS MC,LVR,-33.891788,151.176251
289,1211,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251
290,1212,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251


In [7]:
nsw1_df.shape

(1764, 7)

Second step is to get the boundary of south of Sydney, suburb above -34 to the north should be selected.

In [8]:
nsw2_df = nsw1_df.loc[nsw1_df['lat'] >= (-33.93)]
nsw2_df.head()

,postcode,suburb,state,dc,type,lat,lon
287,1209,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251
288,1210,AUSTRALIA SQUARE,NSW,EAST SUBS MC,LVR,-33.891788,151.176251
289,1211,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251
290,1212,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251
291,1213,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251


In [9]:
nsw2_df.shape

(286, 7)

To get the west of Sydney as well I need to filter the long as well.

In [10]:
nsw3_df = nsw2_df.loc[nsw2_df['lon'] >= (151.14)]
nsw3_df.head()

,postcode,suburb,state,dc,type,lat,lon
287,1209,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251
288,1210,AUSTRALIA SQUARE,NSW,EAST SUBS MC,LVR,-33.891788,151.176251
289,1211,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251
290,1212,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251
291,1213,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251


In [11]:
nsw3_df.shape

(128, 7)

#### Changing the columns name ####


In [12]:
nsw4_df = nsw3_df.rename(columns={'suburb' : 'Neighbourhood', 'lat' : 'Latitude', 'lon' : 'Longitude'})
nsw4_df.head()

,postcode,Neighbourhood,state,dc,type,Latitude,Longitude
287,1209,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251
288,1210,AUSTRALIA SQUARE,NSW,EAST SUBS MC,LVR,-33.891788,151.176251
289,1211,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251
290,1212,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251
291,1213,AUSTRALIA SQUARE,NSW,NaN,LVR,-33.891788,151.176251


#### Remove the dc and type columns and also drop any row with NaN value ####

In [13]:
nsw5_df = nsw4_df.drop(["dc", "type"], axis=1)
nsw5_df.shape

(128, 5)

In [14]:
nsw5_df.head()

,postcode,Neighbourhood,state,Latitude,Longitude
287,1209,AUSTRALIA SQUARE,NSW,-33.891788,151.176251
288,1210,AUSTRALIA SQUARE,NSW,-33.891788,151.176251
289,1211,AUSTRALIA SQUARE,NSW,-33.891788,151.176251
290,1212,AUSTRALIA SQUARE,NSW,-33.891788,151.176251
291,1213,AUSTRALIA SQUARE,NSW,-33.891788,151.176251


In [15]:
nsw6_df = nsw5_df.dropna()
nsw6_df.shape

(128, 5)

It looks like we dont have any NaN in the dataframe.

In [16]:
nsw6_df.to_csv('sydney.csv')

#### Lets create the final dataframe for my analysis as sydney with only three columns Nighbourhood, Latitude and Lonitude

In [17]:
sydney_df = nsw6_df.drop(["postcode","state"], axis=1).reset_index(drop=True)
sydney_df.head()

,Neighbourhood,Latitude,Longitude
0,AUSTRALIA SQUARE,-33.891788,151.176251
1,AUSTRALIA SQUARE,-33.891788,151.176251
2,AUSTRALIA SQUARE,-33.891788,151.176251
3,AUSTRALIA SQUARE,-33.891788,151.176251
4,AUSTRALIA SQUARE,-33.891788,151.176251


###### Get the geographical coordinates of Sydney

In [18]:
address = 'Sydney, NSW'

geolocator = Nominatim(user_agent="ont_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Sydney are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Sydney are -33.8548157, 151.2164539.


##### Now visualize Sydney's neighbourhoods.

In [19]:
# create map of Toronto using latitude and longitude values
map_sydney = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(sydney_df['Latitude'], sydney_df['Longitude'], sydney_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sydney)  
    
map_sydney

##### We are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [20]:
CLIENT_ID = '5GPUKSCMRUQEQIEZ2OXGPL4Y14UV1WNBKLCEEON3HHHYEKVD' # your Foursquare ID
CLIENT_SECRET = '04YUMWDAHN5BYQR1VYPLMHBHOC0NQEGKAOQKUPFOEZTS03KO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5GPUKSCMRUQEQIEZ2OXGPL4Y14UV1WNBKLCEEON3HHHYEKVD
CLIENT_SECRET:04YUMWDAHN5BYQR1VYPLMHBHOC0NQEGKAOQKUPFOEZTS03KO


In [21]:
sydney_df.loc[0, 'Neighbourhood']

'AUSTRALIA SQUARE'

##### Get the Lat and Long of first neighbourhood

In [22]:
neighborhood_latitude = sydney_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = sydney_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = sydney_df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of AUSTRALIA SQUARE are -33.891788, 151.176251.


In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

###### Send the http get request and ge the result from API contrcat as json file. 

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ca33ad2db04f507f821a85e'},
 'response': {'headerLocation': 'Newtown',
  'headerFullLocation': 'Newtown, Sydney',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': -33.887287995499996,
    'lng': 151.18166196806968},
   'sw': {'lat': -33.8962880045, 'lng': 151.17084003193034}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ed184d68231b9ef89f852b8',
       'name': 'Gather On The Green',
       'location': {'address': '15 Fowler St.',
        'crossStreet': 'Australia St.',
        'lat': -33.89026610403827,
        'lng': 151.17618773680476,
        'labeledLatLngs': [{'label': 'display',
          'lat': -33.89026610403827,
          'lng': 151.17618773680476}],
        'distan

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

###### Change the json file into pandas dataframe

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Gather On The Green,Café,-33.890266,151.176188
1,Camperdown Memorial Rest Park,Park,-33.893836,151.178177
2,Store Espresso,Café,-33.890155,151.176137
3,Little Lord,Café,-33.891560,151.172532
4,Acre Farm and Eatery,Restaurant,-33.888066,151.176885


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


## Exporing neighbourhoods in Sydney
A function that looks at all neighbourhoods in Sydney and other cities as well

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
sydney_venues = getNearbyVenues(names=sydney_df['Neighbourhood'],
                                   latitudes=sydney_df['Latitude'],
                                   longitudes=sydney_df['Longitude']
                                  )

AUSTRALIA SQUARE
AUSTRALIA SQUARE
AUSTRALIA SQUARE
AUSTRALIA SQUARE
AUSTRALIA SQUARE
AUSTRALIA SQUARE
AUSTRALIA SQUARE
ROYAL EXCHANGE
ROYAL EXCHANGE
ROYAL EXCHANGE
ROYAL EXCHANGE
ROYAL EXCHANGE
QUEEN VICTORIA BUILDING
QUEEN VICTORIA BUILDING
QUEEN VICTORIA BUILDING
QUEEN VICTORIA BUILDING
QUEEN VICTORIA BUILDING
DARLINGHURST
BONDI JUNCTION
DOUBLE BAY
BROADWAY
EVELEIGH
MASCOT
KENSINGTON
UNSW SYDNEY
DRUMMOYNE
MARRICKVILLE
MARRICKVILLE
MILSONS POINT
ST MARYS
DAWES POINT
HAYMARKET
MILLERS POINT
PARLIAMENT HOUSE
SYDNEY
SYDNEY SOUTH
THE ROCKS
THE UNIVERSITY OF SYDNEY
BROADWAY
ULTIMO
CHIPPENDALE
DARLINGTON
PYRMONT
DARLINGHURST
SURRY HILLS
ELIZABETH BAY
HMAS KUTTABUL
POTTS POINT
RUSHCUTTERS BAY
WOOLLOOMOOLOO
ALEXANDRIA
BEACONSFIELD
EVELEIGH
REDFERN
WATERLOO
ZETLAND
EASTLAKES
ROSEBERY
MOORE PARK
PADDINGTON
BONDI JUNCTION
BONDI JUNCTION PLAZA
QUEENS PARK
BELLEVUE HILL
BRONTE
WAVERLEY
WOOLLAHRA
BONDI
BONDI BEACH
NORTH BONDI
TAMARAMA
DARLING POINT
EDGECLIFF
HMAS RUSHCUTTERS
POINT PIPER
DOUBLE BAY


##### The size of the new dataframe

In [30]:
print(sydney_venues.shape)
sydney_venues.head()

(4518, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,AUSTRALIA SQUARE,-33.891788,151.176251,Gather On The Green,-33.890266,151.176188,Café
1,AUSTRALIA SQUARE,-33.891788,151.176251,Camperdown Memorial Rest Park,-33.893836,151.178177,Park
2,AUSTRALIA SQUARE,-33.891788,151.176251,Store Espresso,-33.890155,151.176137,Café
3,AUSTRALIA SQUARE,-33.891788,151.176251,Little Lord,-33.891560,151.172532,Café
4,AUSTRALIA SQUARE,-33.891788,151.176251,Acre Farm and Eatery,-33.888066,151.176885,Restaurant


#### Here are the unmbers of venues in Sydney's neighbourhoods

In [31]:
sydney_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
ALEXANDRIA,25,25,25,25,25,25
ANNANDALE,22,22,22,22,22,22
AUSTRALIA SQUARE,70,70,70,70,70,70
BALMAIN,48,48,48,48,48,48
BALMAIN EAST,7,7,7,7,7,7
BEACONSFIELD,19,19,19,19,19,19
BELLEVUE HILL,7,7,7,7,7,7
BIRCHGROVE,31,31,31,31,31,31
BONDI,15,15,15,15,15,15


In [32]:
print('There are {} uniques categories.'.format(len(sydney_venues['Venue Category'].unique())))

There are 244 uniques categories.


In [33]:
# one hot encoding
sydney_onehot = pd.get_dummies(sydney_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
sydney_onehot['Neighbourhood'] = sydney_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [sydney_onehot.columns[-1]] + list(sydney_onehot.columns[:-1])
sydney_onehot = sydney_onehot[fixed_columns]
sydney_onehot.head()

,Neighbourhood,Advertising Agency,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Rec Center,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lighthouse,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Modern Greek Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nature Preserve,Neighborhood,Nightclub,Noodle House,Office,Organic Grocery,Other Nightlife,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Restaurant,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,South Indian Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Trade School,Trail,Train Station,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,AUSTRALIA SQUARE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,AUSTRALIA SQUARE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Check the new dataframe size


In [34]:
sydney_onehot.shape

(4518, 245)

Grouping row by neighbourhoods and by taking the mean of the frequency of occurrence of each category

In [35]:
sydeny_grouped = sydney_onehot.groupby('Neighbourhood').mean().reset_index()
sydeny_grouped

,Neighbourhood,Advertising Agency,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Rec Center,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lighthouse,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Modern European Restaurant,Modern Greek Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Venue,Nature Preserve,Neighborhood,Nightclub,Noodle House,Office,Organic Grocery,Other Nightlife,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Resort,Restaurant,Road,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soccer Field,South Indian Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Trade School,Trail,Train Station,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Water Park,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio
0,ALEXANDRIA,0.0,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.160000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [36]:
sydeny_grouped.shape

(103, 245)

#### Each neighbourhood with the top 5 most common venues

In [37]:
num_top_venues = 5

for hood in sydeny_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = sydeny_grouped[sydeny_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALEXANDRIA----
                venue  freq
0                 Pub  0.16
1                Café  0.16
2  Italian Restaurant  0.08
3     Thai Restaurant  0.08
4            Dive Bar  0.04


----ANNANDALE----
                   venue  freq
0                   Café  0.36
1          Grocery Store  0.09
2  Australian Restaurant  0.05
3      Fish & Chips Shop  0.05
4                    Bar  0.05


----AUSTRALIA SQUARE----
          venue  freq
0          Café   0.4
1           Pub   0.1
2    Restaurant   0.1
3  Liquor Store   0.1
4          Park   0.1


----BALMAIN----
                 venue  freq
0                 Café  0.27
1                  Pub  0.10
2     Sushi Restaurant  0.06
3          Pizza Place  0.04
4  Japanese Restaurant  0.04


----BALMAIN EAST----
                venue  freq
0                Café  0.29
1  Italian Restaurant  0.14
2       Bowling Green  0.14
3           Gastropub  0.14
4       Shopping Mall  0.14


----BEACONSFIELD----
                venue  freq
0         Coff

            venue  freq
0            Café  0.16
1     Pizza Place  0.07
2     Coffee Shop  0.05
3  Farmers Market  0.05
4    Burger Joint  0.05


----KINGSFORD----
                   venue  freq
0     Chinese Restaurant  0.17
1       Asian Restaurant  0.12
2  Indonesian Restaurant  0.12
3          Grocery Store  0.07
4       Malay Restaurant  0.07


----KIRRIBILLI----
                          venue  freq
0                          Café  0.22
1                          Park  0.08
2            Italian Restaurant  0.08
3  Theme Park Ride / Attraction  0.08
4               Thai Restaurant  0.06


----LAVENDER BAY----
                 venue  freq
0                 Café  0.39
1                  Bar  0.07
2            Juice Bar  0.05
3  Japanese Restaurant  0.05
4                  Pub  0.05


----LEICHHARDT----
                  venue  freq
0                  Café  0.22
1    Italian Restaurant  0.17
2                   Pub  0.05
3  Brazilian Restaurant  0.05
4   Japanese Restaurant  0.05


-

                   venue  freq
0                   Café  0.15
1                   Park  0.12
2                  Beach  0.09
3     Italian Restaurant  0.06
4  Australian Restaurant  0.06


----TEMPE----
         venue  freq
0          Pub  0.14
1  Golf Course  0.14
2         Park  0.14
3     Platform  0.14
4   Food Truck  0.14


----THE ROCKS----
                   venue  freq
0                   Café  0.10
1                  Hotel  0.07
2                    Pub  0.06
3  Australian Restaurant  0.06
4     Italian Restaurant  0.04


----THE UNIVERSITY OF SYDNEY----
            venue  freq
0            Café  0.24
1     Coffee Shop  0.10
2     Pizza Place  0.10
3             Pub  0.05
4  History Museum  0.05


----TURRELLA----
             venue  freq
0             Park   0.2
1  Paintball Field   0.2
2         Platform   0.2
3    Train Station   0.2
4      Bus Station   0.2


----ULTIMO----
                   venue  freq
0                   Café  0.31
1                    Pub  0.08
2    Dum

#### The function below will sort venues in descending order

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Let's create new dataframe by using the above function and call it neighbourhoods_venues_sorted

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = sydeny_grouped['Neighbourhood']

for ind in np.arange(sydeny_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sydeny_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALEXANDRIA,Pub,Café,Thai Restaurant,Italian Restaurant,Market,Office,Coffee Shop,Burger Joint,Bakery,Grocery Store
1,ANNANDALE,Café,Grocery Store,Pub,Bar,Park,Fish & Chips Shop,Australian Restaurant,Asian Restaurant,Pizza Place,Fried Chicken Joint
2,AUSTRALIA SQUARE,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
3,BALMAIN,Café,Pub,Sushi Restaurant,Park,Japanese Restaurant,Pizza Place,Indian Restaurant,Bar,Tapas Restaurant,Beer Garden
4,BALMAIN EAST,Café,Shopping Mall,Italian Restaurant,Bowling Green,Park,Gastropub,Yoga Studio,Field,Fish & Chips Shop,Flea Market


#### Cluster neighbourhoods using k-means to 5 clusters

In [40]:
# set number of clusters
kclusters = 5

sydeny_grouped = sydeny_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sydeny_grouped)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 0, 2, 2, 0, 2], dtype=int32)

#### Add labels to the clusters

In [41]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sydney_merged = sydney_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
sydney_merged = sydney_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how='right')

sydney_merged # check the last columns!

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AUSTRALIA SQUARE,-33.891788,151.176251,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
1,AUSTRALIA SQUARE,-33.891788,151.176251,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
2,AUSTRALIA SQUARE,-33.891788,151.176251,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
3,AUSTRALIA SQUARE,-33.891788,151.176251,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
4,AUSTRALIA SQUARE,-33.891788,151.176251,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
5,AUSTRALIA SQUARE,-33.891788,151.176251,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
6,AUSTRALIA SQUARE,-33.891788,151.176251,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
7,ROYAL EXCHANGE,-33.865330,151.207905,2,Café,Coffee Shop,Hotel,Bar,Cocktail Bar,Sandwich Place,Italian Restaurant,Steakhouse,Speakeasy,Australian Restaurant
8,ROYAL EXCHANGE,-33.865330,151.207905,2,Café,Coffee Shop,Hotel,Bar,Cocktail Bar,Sandwich Place,Italian Restaurant,Steakhouse,Speakeasy,Australian Restaurant
9,ROYAL EXCHANGE,-33.865330,151.207905,2,Café,Coffee Shop,Hotel,Bar,Cocktail Bar,Sandwich Place,Italian Restaurant,Steakhouse,Speakeasy,Australian Restaurant


##### Finally, let's visualize the resulting clusters

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sydney_merged['Latitude'], sydney_merged['Longitude'], sydney_merged['Neighbourhood'], sydney_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examin the clusters
#### Cluster 1

In [43]:
sydney_merged.loc[sydney_merged['Cluster Labels'] == 0, sydney_merged.columns[[0] + list(range(3, sydney_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,QUEEN VICTORIA BUILDING,0,Café,Coffee Shop,Shopping Mall,Cocktail Bar,Bar,Hotel,Bookstore,Japanese Restaurant,Speakeasy,Thai Restaurant
13,QUEEN VICTORIA BUILDING,0,Café,Coffee Shop,Shopping Mall,Cocktail Bar,Bar,Hotel,Bookstore,Japanese Restaurant,Speakeasy,Thai Restaurant
14,QUEEN VICTORIA BUILDING,0,Café,Coffee Shop,Shopping Mall,Cocktail Bar,Bar,Hotel,Bookstore,Japanese Restaurant,Speakeasy,Thai Restaurant
15,QUEEN VICTORIA BUILDING,0,Café,Coffee Shop,Shopping Mall,Cocktail Bar,Bar,Hotel,Bookstore,Japanese Restaurant,Speakeasy,Thai Restaurant
16,QUEEN VICTORIA BUILDING,0,Café,Coffee Shop,Shopping Mall,Cocktail Bar,Bar,Hotel,Bookstore,Japanese Restaurant,Speakeasy,Thai Restaurant
18,BONDI JUNCTION,0,Café,Pizza Place,Coffee Shop,Japanese Restaurant,Australian Restaurant,Shopping Mall,Mexican Restaurant,Electronics Store,Pub,Sushi Restaurant
60,BONDI JUNCTION,0,Café,Pizza Place,Coffee Shop,Japanese Restaurant,Australian Restaurant,Shopping Mall,Mexican Restaurant,Electronics Store,Pub,Sushi Restaurant
25,DRUMMOYNE,0,Café,Convenience Store,Pub,Bakery,Grocery Store,Athletics & Sports,Park,Italian Restaurant,Boat or Ferry,Video Store
106,DRUMMOYNE,0,Café,Convenience Store,Pub,Bakery,Grocery Store,Athletics & Sports,Park,Italian Restaurant,Boat or Ferry,Video Store
26,MARRICKVILLE,0,Vietnamese Restaurant,Café,Bar,Japanese Restaurant,Grocery Store,Greek Restaurant,Gym,Deli / Bodega,Ice Cream Shop,Sandwich Place


In [44]:
sydney_merged.loc[sydney_merged['Cluster Labels'] == 1, sydney_merged.columns[[0] + list(range(3, sydney_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
73,HMAS RUSHCUTTERS,1,Park,Yoga Studio,Event Space,Food Truck,Food Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market


In [45]:
sydney_merged.loc[sydney_merged['Cluster Labels'] == 2, sydney_merged.columns[[0] + list(range(3, sydney_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AUSTRALIA SQUARE,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
1,AUSTRALIA SQUARE,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
2,AUSTRALIA SQUARE,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
3,AUSTRALIA SQUARE,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
4,AUSTRALIA SQUARE,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
5,AUSTRALIA SQUARE,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
6,AUSTRALIA SQUARE,2,Café,Restaurant,Liquor Store,Pub,Park,Dog Run,Record Shop,Yoga Studio,Falafel Restaurant,Farmers Market
7,ROYAL EXCHANGE,2,Café,Coffee Shop,Hotel,Bar,Cocktail Bar,Sandwich Place,Italian Restaurant,Steakhouse,Speakeasy,Australian Restaurant
8,ROYAL EXCHANGE,2,Café,Coffee Shop,Hotel,Bar,Cocktail Bar,Sandwich Place,Italian Restaurant,Steakhouse,Speakeasy,Australian Restaurant
9,ROYAL EXCHANGE,2,Café,Coffee Shop,Hotel,Bar,Cocktail Bar,Sandwich Place,Italian Restaurant,Steakhouse,Speakeasy,Australian Restaurant


In [46]:
sydney_merged.loc[sydney_merged['Cluster Labels'] == 3, sydney_merged.columns[[0] + list(range(3, sydney_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,MASCOT,3,Supermarket,Grocery Store,Food & Drink Shop,Shopping Mall,Farmers Market,Yoga Studio,Falafel Restaurant,Food Truck,Food Service,Food Court
56,EASTLAKES,3,Grocery Store,Supermarket,Farmers Market,Shopping Mall,Electronics Store,Food & Drink Shop,Garden Center,Fast Food Restaurant,Field,Fish & Chips Shop


In [47]:
sydney_merged.loc[sydney_merged['Cluster Labels'] == 4, sydney_merged.columns[[0] + list(range(3, sydney_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,QUEENS PARK,4,Café,Yoga Studio,Fried Chicken Joint,Fountain,Food Truck,Food Service,Food Court,Food & Drink Shop,Flower Shop,Flea Market
89,FOREST LODGE,4,Café,Pub,Playground,Vietnamese Restaurant,Fish & Chips Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Flea Market
